# Preamble

In this notebook, we will use data from three different CSV files:
- `credit_cards.csv`: information such as the expiration date and provider.
- `transactions.csv`: events containing information about when a credit card was used, such as a timestamp, location, and the amount spent. A boolean `fraud_label` variable (True/False) tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

In [1]:
# import libraries
import joblib
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians

import xgboost as xgb
from sklearn.metrics import confusion_matrix, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# specify the window length as "4h"
window_len = '4h'

# specify the url for the data
url = 'https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_data/'

In [3]:
# read the credit_cards.csv file
credit_cards_df = pd.read_csv(url + 'credit_cards.csv')
credit_cards_df

,cc_num,provider,expires
0,4796807885357879,visa,05/23
1,4529266636192966,visa,03/22
2,4922690008243953,visa,02/27
3,4897369589533543,visa,04/22
4,4848518335893425,visa,10/26
...,...,...,...
1995,4473593503484549,visa,01/26
1996,4483607926610254,visa,08/26
1997,4249540384047310,visa,10/22
1998,4325889817240225,visa,03/24


In [4]:
# read the profiles.csv file
profiles_df = pd.read_csv(url + 'profiles.csv', parse_dates=['birthdate'])
profiles_df

,name,sex,mail,birthdate,City,Country,cc_num
0,Catherine Zimmerman,F,valenciajason@hotmail.com,1988-09-20,Bryn Mawr-Skyway,US,4796807885357879
1,Michael Williams,M,brettkennedy@yahoo.com,1977-03-01,Gates-North Gates,US,4529266636192966
2,Jessica Krueger,F,marthacruz@hotmail.com,1947-09-10,Greenfield,US,4922690008243953
3,Ruth Harris,F,james11@yahoo.com,1983-12-27,New City,US,4897369589533543
4,Paul Ashley,M,matthew97@hotmail.com,1974-11-10,Peabody,US,4848518335893425
...,...,...,...,...,...,...,...
1995,Megan Ramirez,F,jmorrison@gmail.com,1924-06-28,Stockbridge,US,4473593503484549
1996,Jessica Gould,F,larrydiaz@yahoo.com,1925-11-07,Port Richmond,US,4483607926610254
1997,Adam Black,M,richardrusso@yahoo.com,1980-09-15,Brentwood Estates,US,4249540384047310
1998,Brooke Robinson,F,cwagner@hotmail.com,1986-11-05,Redding,US,4325889817240225


In [5]:
# read the transactions.csv file
trans_df = pd.read_csv(url + 'transactions.csv', parse_dates=['datetime'])
trans_df

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,42.308650,-83.482160,Canton,US,0
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,33.522530,-117.707550,Laguna Niguel,US,0
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,37.608760,-77.373310,Mechanicsville,US,0
3,6fb3e6beafbb92b8e15827037f603c52,2022-01-01 00:03:24,4814447237003448,Health/Beauty,54.71,43.540720,-116.563460,Nampa,US,0
4,be0b8acc57bfe126a5a392fd99e6ddd1,2022-01-01 00:03:55,4515188652242507,Grocery,59.22,40.245370,-75.649630,Pottstown,US,0
...,...,...,...,...,...,...,...,...,...,...
106015,8bbf3551f3d68a4764a16249e1f5045b,2022-03-24 10:57:02,4032019521897961,Cash Withdrawal,73.08,41.861505,-87.871980,Westchester,US,0
106016,f6e2c222d3c81d34248e0575deb2be33,2022-03-28 11:57:02,4032019521897961,Cash Withdrawal,287.33,41.870982,-87.862855,Westchester,US,0
106017,9dbbe2a7547e1949e137a28c7f3a50a6,2022-04-01 12:57:02,4032019521897961,Cash Withdrawal,53.88,41.877531,-87.865628,Westchester,US,0
106018,fa32d5d268951cd20bfdbf8f7f6ac855,2022-04-05 13:57:02,4032019521897961,Cash Withdrawal,279.73,41.884127,-87.861584,Westchester,US,0


# Feature Engineering

Typical red flags for fraudulent transactions include large transaction volume and high frequency in the span of a few hours. Another regular characteristic is that elderly people are usually targeted by scammers.

To faciliate model learning, we will create two types of features:
1. features that aggregate data from different sources.
    - customer's age at time of transaction
    - credit card's time to expiry
2. features that aggregate data from multiple time stamps
    - transaction frequency in a given window

In [6]:
# merge the trans_df with the profiles_df to get age
age_df = trans_df.merge(profiles_df, on='cc_num', how='left')
age_df

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label,name,sex,mail,birthdate,City,Country
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,42.308650,-83.482160,Canton,US,0,Megan Ramirez,F,jmorrison@gmail.com,1924-06-28,Stockbridge,US
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,33.522530,-117.707550,Laguna Niguel,US,0,George Ryan,M,hsimpson@hotmail.com,1988-04-01,Hutchinson,US
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,37.608760,-77.373310,Mechanicsville,US,0,Jamie Floyd,M,rhondahall@hotmail.com,1941-02-07,Fresno,US
3,6fb3e6beafbb92b8e15827037f603c52,2022-01-01 00:03:24,4814447237003448,Health/Beauty,54.71,43.540720,-116.563460,Nampa,US,0,Yvonne White,F,andrewhickman@hotmail.com,1968-06-23,Pahrump,US
4,be0b8acc57bfe126a5a392fd99e6ddd1,2022-01-01 00:03:55,4515188652242507,Grocery,59.22,40.245370,-75.649630,Pottstown,US,0,Ross Thomas,M,zflores@hotmail.com,1975-12-31,Martinsburg,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106015,8bbf3551f3d68a4764a16249e1f5045b,2022-03-24 10:57:02,4032019521897961,Cash Withdrawal,73.08,41.861505,-87.871980,Westchester,US,0,Tiffany Newman,F,hardinscott@yahoo.com,1973-08-03,Port Richmond,US
106016,f6e2c222d3c81d34248e0575deb2be33,2022-03-28 11:57:02,4032019521897961,Cash Withdrawal,287.33,41.870982,-87.862855,Westchester,US,0,Tiffany Newman,F,hardinscott@yahoo.com,1973-08-03,Port Richmond,US
106017,9dbbe2a7547e1949e137a28c7f3a50a6,2022-04-01 12:57:02,4032019521897961,Cash Withdrawal,53.88,41.877531,-87.865628,Westchester,US,0,Tiffany Newman,F,hardinscott@yahoo.com,1973-08-03,Port Richmond,US
106018,fa32d5d268951cd20bfdbf8f7f6ac855,2022-04-05 13:57:02,4032019521897961,Cash Withdrawal,279.73,41.884127,-87.861584,Westchester,US,0,Tiffany Newman,F,hardinscott@yahoo.com,1973-08-03,Port Richmond,US


In [7]:
# ensure no null values
age_df.isnull().sum()

tid            0
datetime       0
cc_num         0
category       0
amount         0
latitude       0
longitude      0
city           0
country        0
fraud_label    0
name           0
sex            0
mail           0
birthdate      0
City           0
Country        0
dtype: int64

In [8]:
# compute age at the time of each transaction and store it in a newly created column
trans_df['age_at_transaction'] = (age_df['datetime'] - age_df['birthdate']) / np.timedelta64(365, "D")
trans_df.head()

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label,age_at_transaction
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,42.30865,-83.48216,Canton,US,0,97.578083
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,33.52253,-117.70755,Laguna Niguel,US,0,33.775344
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,37.60876,-77.37331,Mechanicsville,US,0,80.953429
3,6fb3e6beafbb92b8e15827037f603c52,2022-01-01 00:03:24,4814447237003448,Health/Beauty,54.71,43.54072,-116.56346,Nampa,US,0,53.561650
4,be0b8acc57bfe126a5a392fd99e6ddd1,2022-01-01 00:03:55,4515188652242507,Grocery,59.22,40.24537,-75.64963,Pottstown,US,0,46.035624


In [9]:
# merge trans_df with credit_cards_df based on cc_num
card_expiry_df = trans_df.merge(credit_cards_df, on='cc_num', how='left')

# change datatype of expires column to datetime format
card_expiry_df['expires'] = pd.to_datetime(card_expiry_df['expires'], format="%m/%y")

# compute days remaining of each credit card at time of transaction
trans_df['days_until_card_expires'] = (card_expiry_df['expires'] - card_expiry_df['datetime']) / np.timedelta64(365,'D')

# display the first few rows of the new features
trans_df[['age_at_transaction','days_until_card_expires']].head()

,age_at_transaction,days_until_card_expires
0,97.578083,4.002739
1,33.775344,4.750683
2,80.953429,0.665749
3,53.561650,0.413692
4,46.035624,1.413691


In the next section we'll create features from aggregations that are computed over every credit card.

In [12]:
# sort trans_df based on the datetime column
trans_df.sort_values('datetime', inplace=True)

# convert the long and lat columns to radians
trans_df[['longitude', 'latitude']] = trans_df[['longitude', 'latitude']].applymap(radians)

trans_df.head()

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label,age_at_transaction,days_until_card_expires
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,0.738425,-1.457039,Canton,US,0,97.578083,4.002739
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,0.585079,-2.054384,Laguna Niguel,US,0,33.775344,4.750683
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,0.656397,-1.350419,Mechanicsville,US,0,80.953429,0.665749
3,6fb3e6beafbb92b8e15827037f603c52,2022-01-01 00:03:24,4814447237003448,Health/Beauty,54.71,0.759929,-2.034416,Nampa,US,0,53.561650,0.413692
4,be0b8acc57bfe126a5a392fd99e6ddd1,2022-01-01 00:03:55,4515188652242507,Grocery,59.22,0.702414,-1.320335,Pottstown,US,0,46.035624,1.413691


In [ ]:
# define function to compute Haversine distance between consecutive coordinates
def haversine(long, lat):
    # shift the long and lat columns to get consecutive values
    long_shifted = long.shift()
    lat_shifted = lat.shift()

    # calculate the difference in long and lat
    long_diff = long_shifted - long
    lat_diff = lat_shifted - lat

    # Haversine formulat to compute distance
    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat) * np.cos(lat_shifted) * np.sin(long_diff/2.0)**2
    c = 2*np.arcsin(np.sqrt(a+b))

    return c